![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [217]:
import pandas as pd

In [218]:
ds_jobs = pd.read_csv("customer_train.csv")

In [219]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [220]:
ds_jobs.memory_usage()

Index                        128
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64

In [221]:
ds_jobs.head(10)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0


### Data Type Conversions

1. Boolean Data Conversion

In [222]:
#possible columns with two values based on the first 10 rows of data
print(ds_jobs["gender"].nunique(), ds_jobs["relevant_experience"].nunique(), ds_jobs["enrolled_university"].nunique(), ds_jobs["major_discipline"].nunique(), ds_jobs["job_change"].nunique())

3 2 3 6 2


In [223]:
#just before boolean conversion, values are converted to 0 and 1 to make a correct transformation
ds_jobs.loc[ds_jobs['relevant_experience'] == 'No relevant experience', 'relevant_experience'] = 0
ds_jobs.loc[ds_jobs['relevant_experience'] == 'Has relevant experience', 'relevant_experience'] = 1

In [224]:
#boolean type conversion
ds_jobs['relevant_experience'] = ds_jobs['relevant_experience'].astype(bool)
ds_jobs['job_change'] = ds_jobs['job_change'].astype(bool)

2. Integer and Float Data Conversion

In [225]:
#although the type of these columns is correct, size reduction is provided
ds_jobs['student_id'] = ds_jobs['student_id'].astype('int32')
ds_jobs['training_hours'] = ds_jobs['training_hours'].astype('int32')
ds_jobs['city_development_index'] = ds_jobs['city_development_index'].astype('float16')

3. Categorical Data Conversion

In [226]:
#categorical data conversion
ds_jobs[['city','gender', 'major_discipline', 'company_type']] = ds_jobs[['city','gender', 'major_discipline', 'company_type']].astype('category')

4. Ordinal Categorical Data Conversion

While the categories in ordinal categorical data have an order within themselves, there is no such hidden information in categorical data. 

In [227]:
#1
ds_jobs['enrolled_university'].unique()

array(['no_enrollment', 'Full time course', nan, 'Part time course'],
      dtype=object)

In [228]:
enrolment = ['no_enrollment', 'Part time course', 'Full time course']
c_enrolment = pd.CategoricalDtype(enrolment, ordered = True)
ds_jobs['enrolled_university'] = ds_jobs['enrolled_university'].astype(c_enrolment)

In [229]:
#2
ds_jobs['education_level'].unique()

array(['Graduate', 'Masters', 'High School', nan, 'Phd', 'Primary School'],
      dtype=object)

In [230]:
ed_lev = ['Primary School','High School', 'Graduate', 'Masters', 'Phd']
c_ed_lev = pd.CategoricalDtype(ed_lev, ordered=True)
ds_jobs['education_level'] = ds_jobs['education_level'].astype(c_ed_lev)

In [231]:
#3
ds_jobs['experience'].unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

In [232]:
exp = list(range(1,21))
exp = [str(i) for i in exp]
exp = ['<1'] + exp + ['>20']

In [233]:
#more than 10 years of experience, this will be used later on to filter the data
more10 = exp[10:]

In [234]:
c_exp = pd.CategoricalDtype(exp, ordered=True)
ds_jobs['experience'] = ds_jobs['experience'].astype(c_exp)

In [235]:
#4
ds_jobs['company_size'].unique()

array([nan, '50-99', '<10', '10000+', '5000-9999', '1000-4999', '10-49',
       '100-499', '500-999'], dtype=object)

In [236]:
size = ['<10', '10-49','50-99','100-499', '500-999','1000-4999','5000-9999', '10000+']  
c_size = pd.CategoricalDtype(size, ordered = True)
ds_jobs['company_size'] = ds_jobs['company_size'].astype(c_size)

In [237]:
#more than 1000 employees, this will be used later on to filter the data
more1000 = size[5:]

In [238]:
#5
ds_jobs['last_new_job'].unique()

array(['1', '>4', 'never', '4', '3', '2', nan], dtype=object)

In [239]:
last = ['never','1', '2', '3', '4', '>4']
c_last = pd.CategoricalDtype(last, ordered=True)
ds_jobs['last_new_job'] = ds_jobs['last_new_job'].astype(c_last)

After all data conversion, filtering was done based on experience level and company size was done.

In [240]:
ds_jobs_transformed = ds_jobs[(ds_jobs['experience'].isin(more10)) &(ds_jobs['company_size'].isin(more1000))]

In [241]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9)

In [242]:
ds_jobs_transformed.memory_usage()

Index                     17608
student_id                 8804
city                       7353
city_development_index     4402
gender                     2333
relevant_experience        2201
enrolled_university        2333
education_level            2413
major_discipline           2421
experience                 2933
company_size               2565
company_type               2421
last_new_job               2421
training_hours             8804
job_change                 2201
dtype: int64